# Model Evaluation


### Import Packages/Libraries + Data + Models


Import libraries/packages


In [ ]:
import sys
import warnings

sys.path.append("../")
from src.data_utils import get_data, get_models
from src.evaluate import evaluate_models
from src.config import BASE_PATH

import pandas as pd

Set Globals


In [ ]:
SAVE_PATH = BASE_PATH / "results"

##Data
OUTCOME_DICT = {
    "surg": get_data("outcome_surg"),
    "bleed": get_data("outcome_bleed"),
    "asp": get_data("outcome_asp"),
    "mort": get_data("outcome_mort"),
}

##Models
model_prefix_list = ["lr", "lgbm", "svc", "stack", "nn"]
model_dir = BASE_PATH / "cal_models"
MODEL_DICT = {}
for outcome in OUTCOME_DICT.keys():
    MODEL_DICT[outcome] = get_models(model_prefix_list, outcome, file_dir=model_dir)

Import base models


In [ ]:
# from src.nn_models import load_nn_clf
# import torch
# DEVICE = 'mps' if torch.backends.mps.is_available() else 'cpu'
# model_prefix_list = ['lr', 'lgbm', 'svc', 'stack']
# ##Can use any X df for input dimension, all = # of features
# nn_in_dim = OUTCOME_DICT['surg']['X_train'].shape[1]
# model_dir = '../models'
# MODEL_DICT = {}
# for outcome in OUTCOME_DICT.keys():
#     MODEL_DICT[outcome] = get_models(model_prefix_list, outcome)
#     MODEL_DICT[outcome]['nn'] = load_nn_clf(data_path= f'{model_dir}/{outcome}/nn.pt',
#                                             in_dim=nn_in_dim,
#                                             device=DEVICE)

### Evaluate

1. AUROC
   - Train vs validation vs test plot
   - 95% CI
2. Confusion Matrices
3. Discrimination Values (w/ 95% CIs)
   - Accuracy
   - f1
   - recall
   - precision
4. Calibration
   - Calibration Curves
   - ICC (w/ 95% CIs)
   - brier score (w/ 95% CIs)


In [ ]:
for outcome, cur_data in OUTCOME_DICT.items():
    print(f"{'*' *30} Outcome: {outcome} {'*' *30}")
    cur_models_dict = MODEL_DICT[outcome]
    class_report_dict = evaluate_models(
        model_dict=cur_models_dict,
        outcome_name=outcome,
        X_train=cur_data["X_train"],
        y_train=cur_data["y_train"].values.ravel(),
        X_val=cur_data["X_val"],
        y_val=cur_data["y_val"].values.ravel(),
        X_test=cur_data["X_test"],
        y_test=cur_data["y_test"].values.ravel(),
        results_path=SAVE_PATH,
        threshold_str="val",
        show_cm=False,
        show_roc=False,
        show_cal=False,
    )
    for ds, report_per_model in class_report_dict.items():
        model_report_df = pd.DataFrame(report_per_model).T
        report_path = SAVE_PATH / "tables" / "class_report" / outcome / f"{ds}.csv"
        if report_path.exists():
            warnings.warn(f"Over-writing class report table at path: {report_path}")
            report_path.unlink()
        report_path.parent.mkdir(exist_ok=True, parents=True)
        model_report_df.to_csv(report_path)